In [1]:
import pandas as pd

In [61]:
df = pd.read_excel("data.xlsx")
df = df.drop(["Age", "Patrol", "Invested"], axis=1)
df = pd.melt(df, id_vars=["Name"], var_name="Requirement", value_name="Passed")
df.loc[df["Passed"] == "X", "Passed"] = True


In [65]:
df

,Name,Requirement,Passed,Level,Theme
0,Thomas Roomes,Promise & Law,True,Membership,
1,Ephraim Rethman,Promise & Law,True,Membership,
2,Matthew Kenyon-Wimbush,Promise & Law,True,Membership,
3,Aiden Marsh,Promise & Law,True,Membership,
4,Annabelle Deering,Promise & Law,True,Membership,
...,...,...,...,...,...
6025,Bridget Halse,None,NaN,End,None
6026,Juliette Marsh,None,NaN,End,None
6027,Logan Le Roux,None,NaN,End,None
6028,Malaika Kazadi,None,NaN,End,None


In [63]:
df["Name"] = df["Name"].apply(lambda x: x[0:x.find("\n")])

In [64]:
df[["Level", "Requirement", "Theme"]] = df["Requirement"].str.split("\n", 2, expand=True)

In [17]:
new_cols = []

for col in df.T.columns:
    new_cols.append(col[0:col.find("\n")])



In [18]:

new_cols

['Thomas Roomes',
 'Ephraim Rethman',
 'Matthew Kenyon-Wimbush',
 'Aiden Marsh',
 'Annabelle Deering',
 'Cameron Block',
 'Julia Webb',
 'Kiran Camis-Bateman',
 'Rachel Cilliers',
 'Timothy Begg',
 'Timothy van Zyl-Smit',
 'Kupakwashe Muringai',
 'Laura Owens',
 'Laura Hoffa',
 'Tristan Staak',
 'Alex Clark',
 'Holly Raine',
 'Joel Foord',
 'Matthew DArcy-Evans',
 'Oliver Hewson',
 'Oliver Van Zyl Smit',
 'Zoë Halse',
 'Nina Starke',
 'Thomas Cilliers',
 'Ado Vandenbosch',
 'Emily Gilham',
 'Jason Franciscus',
 'Mackenzie Block',
 'Oliver Ringrose',
 'Luke Foord',
 'Jack Craddock',
 'Avila van Bever Donker',
 'Bianca Dheda',
 'James Webb',
 'Juliet DArcy- Evans',
 'Rhys Eveleigh',
 'Simon Halse',
 'Alex Marshall',
 'Ethan Gilham',
 'Aidan Block',
 'Bridget Halse',
 'Juliette Marsh',
 'Logan Le Roux',
 'Malaika Kazadi',
 'Oliver Curling']